In [7]:
#install adapter
!pip install azure-storage-blob
!pip install boto3

In [8]:
import json
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
import pandas as pd

In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define airport locations with latitude and longitude
airport_locations = {
'FLA':{"latitude":1.5894336, "longitude":-75.5647457963132},
'RIW':{"latitude":43.0643932499999, "longitude":-108.458996720831},
'ELM':{"latitude":42.16082385, "longitude":-76.8941293224737},
'YKM':{"latitude":46.5668101499999, "longitude":-120.53679819122},
'JFK':{"latitude":40.6429478999999, "longitude":-73.7793733748521},
'GSP':{"latitude":34.8954456, "longitude":-82.2172284565189},
'ATL':{"latitude":33.63779875, "longitude":-84.4292711858567},
'FLL':{"latitude":26.072017, "longitude":-80.1509967313521},
'SPS':{"latitude":33.9661198999999, "longitude":-98.4898970893739},
'SFB':{"latitude":28.7759519, "longitude":-81.2412390376561},
'BUF':{"latitude":42.9402016, "longitude":-78.7307522974535},
'ABE':{"latitude":40.6511631, "longitude":-75.4422538683829},
'DAY':{"latitude":39.9053186999999, "longitude":-84.2202454248026},
'CRW':{"latitude":38.37294755, "longitude":-81.5942197333228},
'KOA':{"latitude":19.743906, "longitude":-156.04229598122},
'GRK':{"latitude":31.0616542, "longitude":-97.8206033626794},
'DAB':{"latitude":29.1816438999999, "longitude":-81.0542694315442},
'CPR':{"latitude":42.9054243, "longitude":-106.464527619496},
'MFE':{"latitude":26.17627985, "longitude":-98.2396054828859},
'DLH':{"latitude":46.84188665, "longitude":-92.2029753758525},
'CWA':{"latitude":44.7727172, "longitude":-89.6671649441226},
'IMT':{"latitude":45.8213995, "longitude":-88.1131141618846},
'SRQ':{"latitude":27.39598835, "longitude":-82.5540742306452},
'FLO':{"latitude":34.18512895, "longitude":-79.7253999100574},
'PWM':{"latitude":43.6461776, "longitude":-70.314411973456},
'ABR':{"latitude":45.446144, "longitude":-98.4206490345065},
'HYS':{"latitude":38.8454764, "longitude":-99.2730373},
'BFM':{"latitude":30.6267668, "longitude":-88.0703012402339},
'MSO':{"latitude":46.91686665, "longitude":-114.087800538914},
'DSM':{"latitude":41.5345492999999, "longitude":-93.6592394458973},
'NCT':{"latitude":10.13928775, "longitude":-85.4445157575337},
'JLN':{"latitude":37.15928825, "longitude":-94.5009028725432},
'GNV':{"latitude":29.69230675, "longitude":-82.2760005501372},
'BTR':{"latitude":30.5319743, "longitude":-91.1504241505103},
'BWI':{"latitude":39.1828341, "longitude":-76.6698273},
'SCK':{"latitude":37.8943699, "longitude":-121.243634558705},
'GSO':{"latitude":36.1020574, "longitude":-79.9445969535714},
'UTA':{"latitude":39.3290743, "longitude":-111.626154720519},
'PIH':{"latitude":42.9074805, "longitude":-112.592839286073},
'HNL':{"latitude":21.32040435, "longitude":-157.917313519816},
'SJT':{"latitude":31.35468635, "longitude":-100.498624549388},
'BZN':{"latitude":45.78359005, "longitude":-111.156979627834},
'CGI':{"latitude":37.2247158499999, "longitude":-89.5680031784597},
'CLL':{"latitude":30.58822845, "longitude":-96.3641116417029},
'EWR':{"latitude":40.68906405, "longitude":-74.1772548503534},
'TXC':{"latitude":54.4417623, "longitude":30.3034005649902},
'MCO':{"latitude":28.41290355, "longitude":-81.3094430975275},
'LYH':{"latitude":37.3221731999999, "longitude":-79.2066029700488},
'LRD':{"latitude":27.54721475, "longitude":-99.4614739871381},
'SBY':{"latitude":38.33941605, "longitude":-75.5101049553024},
'SNA':{"latitude":33.6742055999999, "longitude":-117.868024297016},
'TVC':{"latitude":44.7417370999999, "longitude":-85.5802534892588},
'PAH':{"latitude":37.0596, "longitude":-88.773},
'EUG':{"latitude":44.1241627499999, "longitude":-123.214917964354},
'ROC':{"latitude":43.1161699, "longitude":-77.6726933897961},
'BIL':{"latitude":45.8092348499999, "longitude":-108.547675178859},
'BRL':{"latitude":40.779453, "longitude":-91.1233790861529},
'MCI':{"latitude":39.3023103, "longitude":-94.7211899174345},
'PBI':{"latitude":26.6859478, "longitude":-80.0924865071716},
'MIC':{"latitude":45.06193745, "longitude":-93.352502343064},
'AKA':{"latitude":32.75565235, "longitude":108.881296782116},
'BPT':{"latitude":29.95237505, "longitude":-94.0217808135379},
'DUJ':{"latitude":41.1795228, "longitude":-78.8986431},
'MLU':{"latitude":32.5111077, "longitude":-92.0384615},
'XNA':{"latitude":36.27981, "longitude":-94.3025901659567},
'RUT':{"latitude":43.53221595, "longitude":-72.9504429983352},
'ORF':{"latitude":36.8946697, "longitude":-76.20229629965},
'CRP':{"latitude":27.7702374, "longitude":-97.5065753547286},
'ALW':{"latitude":46.0940856999999, "longitude":-118.291219211658},
'GCC':{"latitude":44.34685815, "longitude":-105.537926180909},
'AUS':{"latitude":30.19348925, "longitude":-97.6650096352967},
'TSA':{"latitude":25.06647695, "longitude":121.554887744347},
'AZO':{"latitude":42.2307865, "longitude":-85.5510419237961},
'COF':{"latitude":28.2380932, "longitude":-80.6079035},
'NJA':{"latitude":35.4544705499999, "longitude":139.442547464513},
'GLH':{"latitude":33.4865987, "longitude":-90.9887059080954},
'ILG':{"latitude":39.6796715, "longitude":-75.6076335419448},
'TYR':{"latitude":32.3519096499999, "longitude":-95.4020875613132},
'NYA':{"latitude":22.3160874, "longitude":113.93647839996},
'PIB':{"latitude":31.4672, "longitude":-89.3363366},
'SFO':{"latitude":37.6224519999999, "longitude":-122.383989385483},
'DDC':{"latitude":37.7670112, "longitude":-99.964318660498},
'VCT':{"latitude":28.8540778, "longitude":-96.918120680313},
'CAK':{"latitude":40.9152061, "longitude":-81.4399241932808},
'VRB':{"latitude":27.6572149499999, "longitude":-80.4183298024738},
'MKL':{"latitude":35.6014507, "longitude":-88.9161165},
'SGU':{"latitude":37.0342442, "longitude":-113.509016545502},
'JST':{"latitude":40.31459165, "longitude":-78.8335324922627},
'OTZ':{"latitude":66.8836843, "longitude":-162.610746610163},
'ARA':{"latitude":30.0330182499999, "longitude":-91.8819074288119},
'SAN':{"latitude":32.73336075, "longitude":-117.192246412776},
'HIB':{"latitude":47.38500865, "longitude":-92.8387487759028},
'AAF':{"latitude":29.72834225, "longitude":-85.0245864699153},
'CNY':{"latitude":38.7689886999999, "longitude":-109.750797439097},
'EGE':{"latitude":39.6425635, "longitude":-106.917134720113},
'MCW':{"latitude":43.16076005, "longitude":-93.327731997609},
'DHN':{"latitude":31.321627, "longitude":-85.4496135106721},
'GRR':{"latitude":42.8766102999999, "longitude":-85.5362909157561},
'LNK':{"latitude":40.84731465, "longitude":-96.7574425595823},
'RKS':{"latitude":41.59507585, "longitude":-109.068421430159},
'ERI':{"latitude":42.0847103499999, "longitude":-80.1744723513412},
'TUL':{"latitude":36.2010229, "longitude":-95.8868134711857},
'GJT':{"latitude":39.1234158, "longitude":-108.524449203066},
'IFP':{"latitude":35.154391, "longitude":-114.558682141203},
'SJU':{"latitude":18.44319945, "longitude":-65.9974303684746},
'EKO':{"latitude":40.82550195, "longitude":-115.788011129418},
'IAH':{"latitude":29.9841416, "longitude":-95.3329859561449},
'CHA':{"latitude":35.03238355, "longitude":-85.2016778878584},
'PUB':{"latitude":38.29844685, "longitude":-104.503307959448},
'MSN':{"latitude":43.1420815, "longitude":-89.3377453366062},
'CAA':{"latitude":14.87529595, "longitude":-85.7749729489726},
'INL':{"latitude":48.5661049, "longitude":-93.4040763804481},
'SGF':{"latitude":37.24298375, "longitude":-93.3889140611615},
'COD':{"latitude":44.52075195, "longitude":-109.021853429653},
'CEC':{"latitude":41.7814506, "longitude":-124.238071084856},
'PAE':{"latitude":47.9088268, "longitude":-122.279430204826},
'ADK':{"latitude":51.8759083, "longitude":-176.650397273742},
'PNS':{"latitude":30.4725975, "longitude":-87.1880859291909},
'PHL':{"latitude":39.875018, "longitude":-75.2352127896966},
'PVU':{"latitude":40.2192141, "longitude":-111.720573768685},
'ALS':{"latitude":35.8210781, "longitude":-77.3790233},
'MAF':{"latitude":31.9460260999999, "longitude":-102.210618682157},
'ELP':{"latitude":31.809584, "longitude":-106.366598029979},
'RSW':{"latitude":26.53294975, "longitude":-81.7588056261221},
'BJI':{"latitude":47.5101068, "longitude":-94.929824346256},
'BOS':{"latitude":42.36317665, "longitude":-71.0136400626945},
'JNU':{"latitude":58.3566191, "longitude":-134.580193001753},
'BOI':{"latitude":43.55988165, "longitude":-116.207822530981},
'ATW':{"latitude":44.25659495, "longitude":-88.521695987557},
'SAT':{"latitude":29.5334021, "longitude":-98.4692144304692},
'HVN':{"latitude":41.2626111, "longitude":-72.8856354728354},
'GDV':{"latitude":47.13812935, "longitude":-104.808643327326},
'LAR':{"latitude":41.3134545, "longitude":-105.672865393546},
'ACV':{"latitude":40.9763645, "longitude":-124.108425899374},
'MGW':{"latitude":39.642736, "longitude":-79.9192332875},
'OTH':{"latitude":43.41549325, "longitude":-124.243465416691},
'PLN':{"latitude":45.56926645, "longitude":-84.7969503049933},
'DCA':{"latitude":38.8512894499999, "longitude":-77.0396889391337},
'SHD':{"latitude":38.26252605, "longitude":-78.8999640708124},
'JAN':{"latitude":32.3121103999999, "longitude":-90.0767782942324},
'PRC':{"latitude":22.3160874, "longitude":113.93647839996},
'MTJ':{"latitude":38.5118206, "longitude":-107.892487956184},
'PHX':{"latitude":33.43284865, "longitude":-112.006791456578},
'ITH':{"latitude":42.49180205, "longitude":-76.4606946488083},
'BHM':{"latitude":33.56350085, "longitude":-86.751516067555},
'PPG':{"latitude":-14.3347658, "longitude":-170.715968627237},
'EAR':{"latitude":40.7266522, "longitude":-99.0016625000111},
'ANC':{"latitude":61.18103525, "longitude":-149.997891897304},
'OME':{"latitude":64.5124643, "longitude":-165.456015198387},
'TUS':{"latitude":32.11264765, "longitude":-110.934637645379},
'MKE':{"latitude":42.9465423, "longitude":-87.8972577871441},
'AIA':{"latitude":42.0550551, "longitude":-102.808705764248},
'WAA':{"latitude":65.6230160499999, "longitude":-168.094803322691},
'MIA':{"latitude":25.7949789, "longitude":-80.286723410954},
'CMI':{"latitude":40.0383172, "longitude":-88.2715668483221},
'HOB':{"latitude":32.68632, "longitude":-103.214294942436},
'BFF':{"latitude":41.87685875, "longitude":-103.594918455729},
'DEC':{"latitude":39.83349835, "longitude":-88.8619910050873},
'MCK':{"latitude":40.20704925, "longitude":-100.591657584023},
'OWB':{"latitude":37.73827615, "longitude":-87.164491564207},
'FAY':{"latitude":34.9921371, "longitude":-78.8841682052182},
'GGG':{"latitude":32.38381895, "longitude":-94.7136635767293},
'BIS':{"latitude":46.7732585, "longitude":-100.747163666215},
'PAX':{"latitude":19.93488935, "longitude":-72.8467886330522},
'FNT':{"latitude":42.9607553, "longitude":-83.745631779992},
'FAI':{"latitude":64.8144321, "longitude":-147.858591296532},
'TPA':{"latitude":27.9791649, "longitude":-82.5349276153517},
'COS':{"latitude":38.8004299, "longitude":-104.703277086999},
'EAT':{"latitude":47.39918335, "longitude":-120.205690253463},
'MFR':{"latitude":42.3727441, "longitude":-122.873438344172},
'FSM':{"latitude":35.3392525, "longitude":-94.35697826263},
'RHI':{"latitude":45.6314535, "longitude":-89.478974050743},
'HYA':{"latitude":41.6707658, "longitude":-70.2843904252689},
'WRG':{"latitude":56.4839528, "longitude":-132.368897392567},
'CVG':{"latitude":39.0499064499999, "longitude":-84.6651574942926},
'CHK':{"latitude":-22.3546686, "longitude":119.64224385424},
'TVF':{"latitude":48.0660335, "longitude":-96.1844694},
'CEZ':{"latitude":37.3043669, "longitude":-108.6276045},
'CMX':{"latitude":47.16827015, "longitude":-88.4865113023142},
'MGM':{"latitude":32.2962424, "longitude":-86.3904185136211},
'SWO':{"latitude":36.16188335, "longitude":-97.0874680653493},
'SWF':{"latitude":41.5020532, "longitude":-74.1212028801658},
'LFT':{"latitude":30.2031617, "longitude":-91.9899106562817},
'BRW':{"latitude":71.28622155, "longitude":-156.774570767108},
'FAR':{"latitude":46.9208925, "longitude":-96.8144880299696},
'PKB':{"latitude":39.3434644, "longitude":-81.4401026346384},
'CDV':{"latitude":60.49175165, "longitude":-145.473885357137},
'DRO':{"latitude":37.15284235, "longitude":-107.758605917399},
'MRY':{"latitude":36.58705795, "longitude":-121.841939784365},
'HTS':{"latitude":38.36796505, "longitude":-82.5596021189774},
'TUP':{"latitude":34.26748865, "longitude":-88.7684748641638},
'RST':{"latitude":43.9043097499999, "longitude":-92.4954441252915},
'FOD':{"latitude":42.55024315, "longitude":-94.1932797730044},
'IND':{"latitude":39.7162533, "longitude":-86.2950838853997},
'MVY':{"latitude":41.3923919, "longitude":-70.6145956897731},
'ROW':{"latitude":33.3006936, "longitude":-104.5316574},
'SLN':{"latitude":38.7927540999999, "longitude":-97.6548660826017},
'LSE':{"latitude":43.878607, "longitude":-91.2555715354231},
'CDC':{"latitude":37.7025428, "longitude":-113.099102215117},
'SBD':{"latitude":34.09599685, "longitude":-117.234569101413},
'MDT':{"latitude":40.1932098, "longitude":-76.7529458728774},
'BLV':{"latitude":38.5461016, "longitude":-89.8199744606753},
'BIH':{"latitude":37.3728747, "longitude":-118.365749448212},
'LBB':{"latitude":33.66525745, "longitude":-101.826688667399},
'SJC':{"latitude":37.3633299, "longitude":-121.929336526025},
'MYR':{"latitude":33.6806406, "longitude":-78.9290597087289},
'ROA':{"latitude":37.32442625, "longitude":-79.9767973923541},
'PIE':{"latitude":27.90994325, "longitude":-82.6877178936921},
'JAC':{"latitude":43.6087926, "longitude":-110.738032484947},
'JMS':{"latitude":46.9276783, "longitude":-98.6781353070391},
'DTW':{"latitude":42.2056991, "longitude":-83.3529753762165},
'MOT':{"latitude":48.25794645, "longitude":-101.277005043468},
'KCM':{"latitude":37.53790305, "longitude":36.9542083138203},
'GEG':{"latitude":47.62004405, "longitude":-117.542366655138},
'STL':{"latitude":38.74962725, "longitude":-90.3704702495208},
'LAN':{"latitude":42.77810185, "longitude":-84.5858648892843},
'GRI':{"latitude":40.9705149499999, "longitude":-98.3075367171},
'ABY':{"latitude":31.5339678, "longitude":-84.1925449341479},
'PSP':{"latitude":33.83033915, "longitude":-116.509645345311},
'GUM':{"latitude":13.4864901999999, "longitude":144.802060253525},
'GTR':{"latitude":33.4502, "longitude":-88.5903986},
'TEX':{"latitude":31.5817032, "longitude":-102.8960083},
'OHT':{"latitude":33.57070845, "longitude":71.4430210024095},
'EYW':{"latitude":24.5554773, "longitude":-81.7596160347507},
'RDD':{"latitude":40.51108035, "longitude":-122.296109047536},
'TBN':{"latitude":37.739739, "longitude":-92.1391957940205},
'HXD':{"latitude":37.1238005999999, "longitude":97.263388131233},
'DEN':{"latitude":39.8606676, "longitude":-104.685367326102},
'COU':{"latitude":38.8168307, "longitude":-92.2198931},
'ICT':{"latitude":37.649253, "longitude":-97.4322633030929},
'NJC':{"latitude":60.9494543499999, "longitude":76.4856218756399},
'TOL':{"latitude":-4.98093375, "longitude":152.010079392859},
'AVL':{"latitude":35.4364233, "longitude":-82.5408809504962},
'EAU':{"latitude":44.8689683, "longitude":-91.4855007489738},
'SAV':{"latitude":32.12423765, "longitude":-81.1977064477444},
'ORD':{"latitude":41.9780813, "longitude":-87.909181769304},
'MKG':{"latitude":43.1703365, "longitude":-86.2370531},
'ACP':{"latitude":37.3477903999999, "longitude":46.1241048979673},
'DIK':{"latitude":46.80199425, "longitude":-102.80051985071},
'WYS':{"latitude":44.6889409, "longitude":-111.116883943111},
'LBF':{"latitude":41.12802075, "longitude":-100.688624423615},
'KTN':{"latitude":55.3537647499999, "longitude":-131.708356328807},
'HOU':{"latitude":29.64714905, "longitude":-95.2769303131995},
'SBN':{"latitude":41.7091043, "longitude":-86.3232534086752},
'SEA':{"latitude":47.4475673, "longitude":-122.308015856951},
'BGR':{"latitude":44.8072619, "longitude":-68.8194148610511},
'SMX':{"latitude":34.8994719, "longitude":-120.455516179733},
'PIA':{"latitude":-27.1891033, "longitude":116.2846248},
'LBL':{"latitude":37.0449620499999, "longitude":-100.958667137454},
'DAL':{"latitude":32.84524055, "longitude":-96.8473218756906},
'CLE':{"latitude":41.40645365, "longitude":-81.8511010383639},
'CHS':{"latitude":32.89929845, "longitude":-80.0394650305042},
'SPI':{"latitude":39.8436861499999, "longitude":-89.6795659461597},
'HSV':{"latitude":34.63699305, "longitude":-86.7638338837723},
'AVP':{"latitude":41.33709205, "longitude":-75.7225712292862},
'CVN':{"latitude":34.42584135, "longitude":-103.079707675426},
'LNS':{"latitude":40.12190825, "longitude":-76.2975126607952},
'LWB':{"latitude":37.8586232, "longitude":-80.3996371545226},
'ALO':{"latitude":42.5577097, "longitude":-92.3940615771661},
'MSY':{"latitude":29.99420015, "longitude":-90.2643043936697},
'NCA':{"latitude":21.9157059, "longitude":-71.9443193415783},
'PIR':{"latitude":44.37841305, "longitude":-100.282561872904},
'ABI':{"latitude":61.62916165, "longitude":-149.046954243994},
'PGD':{"latitude":26.9194137, "longitude":-81.9930281031229},
'AKN':{"latitude":58.6753166, "longitude":-156.649387025181},
'TWF':{"latitude":42.4817, "longitude":-114.4877},
'RIC':{"latitude":37.5073698, "longitude":-77.3336845},
'ACY':{"latitude":39.45708535, "longitude":-74.570276490922},
'CAE':{"latitude":33.93886305, "longitude":-81.1139715103596},
'MWA':{"latitude":37.75961685, "longitude":-89.0120349134521},
'GPT':{"latitude":22.3160874, "longitude":113.93647839996},
'RDM':{"latitude":44.25408195, "longitude":-121.14906868928},
'TRI':{"latitude":36.47344765, "longitude":-82.4084326869909},
'CLT':{"latitude":35.21074145, "longitude":-80.9457435226013},
'SAW':{"latitude":40.89433405, "longitude":29.3118058411142},
'BFD':{"latitude":41.8034637, "longitude":-78.6403028202179},
'STT':{"latitude":18.33476905, "longitude":-64.9724536459818},
'XWA':{"latitude":48.2605484, "longitude":-103.746770728767},
'ISP':{"latitude":40.79580915, "longitude":-73.0992008580396},
'BFL':{"latitude":35.4364902, "longitude":-119.058883797881},
'NVA':{"latitude":2.95270075, "longitude":-75.2933310117559},
'AUG':{"latitude":44.3220048, "longitude":-69.7983175999799},
'LBE':{"latitude":40.2729348, "longitude":-79.406419571798},
'SLK':{"latitude":44.38940195, "longitude":-74.204655569888},
'GCK':{"latitude":37.92967115, "longitude":-100.726431892871},
'GAA':{"latitude":9.15, "longitude":-74.233333},
'AEX':{"latitude":31.3292736499999, "longitude":-92.5409548662772},
'MCN':{"latitude":32.6938394999999, "longitude":-83.6480177704344},
'PBG':{"latitude":44.6516866, "longitude":-73.4643666977021},
'PSE':{"latitude":18.0088859999999, "longitude":-66.5633497464292},
'HVR':{"latitude":48.543, "longitude":-109.7624},
'SYR':{"latitude":43.1141752, "longitude":-76.119144511436},
'OGG':{"latitude":20.89569785, "longitude":-156.434047641227},
'STX':{"latitude":17.69997805, "longitude":-64.8059208987329},
'ONT':{"latitude":34.05414245, "longitude":-117.59915769373},
'IPL':{"latitude":32.83500895, "longitude":-115.574282700659},
'RAP':{"latitude":44.0448069, "longitude":-103.060680124456},
'RFD':{"latitude":42.1966203, "longitude":-89.10185948684},
'LGA':{"latitude":40.7757145, "longitude":-73.8733639851154},
'VPS':{"latitude":30.4954202, "longitude":-86.549503},
'BQN':{"latitude":18.49346055, "longitude":-67.1342158661154},
'TLH':{"latitude":30.39782605, "longitude":-84.3590839436852},
'UIN':{"latitude":39.9447505, "longitude":-91.1937894178547},
'FLG':{"latitude":35.14039825, "longitude":-111.67020441781},
'OAK':{"latitude":37.72225915, "longitude":-122.217016615532},
'MLI':{"latitude":41.44772135, "longitude":-90.5114665073562},
'GSN':{"latitude":-31.4596996, "longitude":137.173996},
'BLI':{"latitude":48.79368955, "longitude":-122.535956907611},
'LEX':{"latitude":38.03480425, "longitude":-84.6049217412858},
'SIT':{"latitude":57.0476805499999, "longitude":-135.363193631342},
'PHF':{"latitude":37.1322964999999, "longitude":-76.494012924145},
'BHB':{"latitude":44.4500096, "longitude":-68.3621240886538},
'GFK':{"latitude":47.9518219, "longitude":-97.174470652996},
'TYS':{"latitude":35.8095529999999, "longitude":-83.9988716507707},
'INF':{"latitude":19.56168865, "longitude":5.74979210158223},
'AOO':{"latitude":40.29686675, "longitude":-78.3211743846874},
'PQI':{"latitude":46.6911992999999, "longitude":-68.047610593994},
'GRB':{"latitude":44.47848805, "longitude":-88.1436200715983},
'FSD':{"latitude":43.58273465, "longitude":-96.7412031133591},
'LIT':{"latitude":34.7311092, "longitude":-92.2245781780174},
'MSC':{"latitude":33.4590398, "longitude":-111.729871782214},
'SAF':{"latitude":35.61831735, "longitude":-106.087831607797},
'LWS':{"latitude":46.37573125, "longitude":-117.015503028874},
'ATY':{"latitude":44.9129309, "longitude":-97.1487774114251},
'LGB':{"latitude":33.8155658999999, "longitude":-118.153429221667},
'MBL':{"latitude":44.2717152, "longitude":-86.2406121769047},
'VEL':{"latitude":40.43530255, "longitude":-109.511607163927},
'AGS':{"latitude":33.37230205, "longitude":-81.9650639098103},
'IAF':{"latitude":-10.3696728, "longitude":56.6016888483076},
'YAK':{"latitude":59.50081355, "longitude":-139.644080833562},
'SHR':{"latitude":44.7665812, "longitude":-106.977638546442},
'BKW':{"latitude":37.7945921, "longitude":-81.1222664716566},
'BRO':{"latitude":25.905718, "longitude":-97.4255292250408},
'PSM':{"latitude":43.07902105, "longitude":-70.8186776840623},
'SCP':{"latitude":44.7015847499999, "longitude":6.59979035050325},
'BDL':{"latitude":41.9405835, "longitude":-72.685019942646},
'SBP':{"latitude":35.2374913999999, "longitude":-120.643319929432},
'ECP':{"latitude":30.35993085, "longitude":-85.8040456434623},
'LIH':{"latitude":21.97206585, "longitude":-159.336715383903},
'RNO':{"latitude":39.4986461999999, "longitude":-119.773331860864},
'ITO':{"latitude":19.7188305999999, "longitude":-155.047553914476},
'ORH':{"latitude":42.2686095, "longitude":-71.8730639833944},
'MEI':{"latitude":32.32774155, "longitude":-88.7525782827135},
'MSP':{"latitude":44.87801905, "longitude":-93.2209280537696},
'PIT':{"latitude":40.49178275, "longitude":-80.2370244887374},
'VLD':{"latitude":30.77831025, "longitude":-83.2780341085929},
'SDF':{"latitude":38.1716962, "longitude":-85.7354075638109},
'LAX':{"latitude":33.9421675499999, "longitude":-118.421359061415},
'LAS':{"latitude":36.0861034, "longitude":-115.161119791923},
'LCH':{"latitude":30.1224220499999, "longitude":-93.2220625633995},
'SHV':{"latitude":32.4451121, "longitude":-93.8267371598938},
'EVV':{"latitude":38.04188775, "longitude":-87.5296839642167},
'MEM':{"latitude":35.04625645, "longitude":-89.9768946651318},
'IAD':{"latitude":38.95226625, "longitude":-77.4534849116007},
'BCP':{"latitude":-5.8632507, "longitude":146.493276077818},
'GGW':{"latitude":48.2150307, "longitude":-106.613465},
'DFW':{"latitude":32.89651945, "longitude":-97.0465220537124},
'CSG':{"latitude":32.51660405, "longitude":-84.9420079018137},
'LEB':{"latitude":43.6224249, "longitude":-72.3062552438515},
'GUC':{"latitude":38.5323609, "longitude":-106.939827291343},
'PDX':{"latitude":45.5871326, "longitude":-122.592779849391},
'ASE':{"latitude":39.22142085, "longitude":-106.867341176313},
'SLC':{"latitude":40.7900661, "longitude":-111.979898461855},
'RDU':{"latitude":35.8803614499999, "longitude":-78.7872382171651},
'JAX':{"latitude":30.49375, "longitude":-81.6930381608379},
'BGM':{"latitude":42.2106484, "longitude":-75.9815509339948},
'MDW':{"latitude":41.7854316, "longitude":-87.7508486935714},
'FAT':{"latitude":36.776006, "longitude":-119.719744562093},
'BQK':{"latitude":31.259811, "longitude":-81.4639706620112},
'CMH':{"latitude":39.99705605, "longitude":-82.8930406148434},
'STS':{"latitude":38.51034965, "longitude":-122.814833849446},
'ABQ':{"latitude":35.0376694499999, "longitude":-106.610967521768},
'TNA':{"latitude":36.8572453, "longitude":117.211734102726},
'SUX':{"latitude":42.4036084, "longitude":-96.3836488},
'BTM':{"latitude":45.95485055, "longitude":-112.496595303241},
'MOB':{"latitude":30.69214065, "longitude":-88.2411574906622},
'AMA':{"latitude":-4.1015945, "longitude":141.670323413266},
'CKB':{"latitude":39.29950215, "longitude":-80.2284323131777},
'MBS':{"latitude":43.5357961499999, "longitude":-84.0828495668065},
'EWN':{"latitude":35.0709799499999, "longitude":-77.039500540912},
'BTV':{"latitude":44.4728965, "longitude":-73.1514613664556},
'CID':{"latitude":41.8853041, "longitude":-91.714724183198},
'PUW':{"latitude":46.7406033, "longitude":-117.117392583829},
'DVL':{"latitude":48.1138202, "longitude":-98.9072037},
'BET':{"latitude":60.7789171, "longitude":-161.838115161744},
'TXK':{"latitude":33.45252615, "longitude":-93.9902501942041},
'ACT':{"latitude":31.6127285, "longitude":-97.2248400886628},
'ADQ':{"latitude":57.7535941, "longitude":-152.5136697},
'ACK':{"latitude":41.2558238999999, "longitude":-70.0580377540584},
'GPI':{"latitude":2.5708431, "longitude":-77.8977954730552},
'OKC':{"latitude":35.3955479, "longitude":-97.5962256},
'HDN':{"latitude":40.4844693, "longitude":-107.220936},
'OLF':{"latitude":48.09407365, "longitude":-105.573649237867},
'INE':{"latitude":7.0653108, "longitude":171.5515821},
'OAJ':{"latitude":34.82679245, "longitude":-77.6149339094656},
'HRL':{"latitude":26.23004735, "longitude":-97.6537080091815},
'DBQ':{"latitude":42.4013233999999, "longitude":-90.7064772970826},
'MHT':{"latitude":42.93066555, "longitude":-71.4388086668277},
'RKD':{"latitude":44.06139565, "longitude":-69.1029358153421},
'PGV':{"latitude":35.63797635, "longitude":-77.3800183967476},
'ESC':{"latitude":45.72455915, "longitude":-87.0889932834475},
'CHO':{"latitude":38.1410056, "longitude":-78.4504579791172},
'PGA':{"latitude":36.9265921, "longitude":-111.448467392878},
'SOW':{"latitude":34.26500865, "longitude":-110.010407065141},
'SMF':{"latitude":38.68722345, "longitude":-121.590370139436},
'IRK':{"latitude":40.0924817, "longitude":-92.5432908296509},
'CDR':{"latitude":42.83845375, "longitude":-103.096059786877},
'BRD':{"latitude":46.40507465, "longitude":-94.1282799256842},
'DLG':{"latitude":59.0433568, "longitude":-158.5112028},
'SCC':{"latitude":70.19515995, "longitude":-148.464822158771},
'IDA':{"latitude":43.5151828, "longitude":-112.066109499572},
'TTN':{"latitude":40.2764119499999, "longitude":-74.813211083687},
'OMA':{"latitude":41.3011252, "longitude":-95.8898709146803},
'SDY':{"latitude":47.70467, "longitude":-104.190305928887},
'SBA':{"latitude":34.4274301, "longitude":-119.843139924037},
'CIU':{"latitude":46.2481556, "longitude":-84.4691458},
'GST':{"latitude":58.4237706499999, "longitude":-135.706311324844},
'ART':{"latitude":43.9896298, "longitude":-76.0223670714887},
'ALB':{"latitude":42.7480159499999, "longitude":-73.7985556376393},
'IAG':{"latitude":43.1072662499999, "longitude":-78.9523498225054},
'BNA':{"latitude":36.11958985, "longitude":-86.6830870329184},
'PVD':{"latitude":41.72243995, "longitude":-71.4285882197494},
'HLN':{"latitude":46.6063969, "longitude":-111.982744470103},
'PSG':{"latitude":56.80197975, "longitude":-132.944931054308},
'LAW':{"latitude":34.5698495, "longitude":-98.4181222990062},
'FWA':{"latitude":40.9798288, "longitude":-85.1879116315925},
'PVC':{"latitude":42.0717956999999, "longitude":-70.2233804266807},
'HGR':{"latitude":39.70883285, "longitude":-77.7267563770384},
'MSL':{"latitude":34.7458927499999, "longitude":-87.6115294406162},
'GTF':{"latitude":47.4800263, "longitude":-111.375578974821},
'AZA':{"latitude":33.30591455, "longitude":-111.652644115019},
'BUR':{"latitude":34.20317265, "longitude":-118.359178243885},
'MLB':{"latitude":28.1027201, "longitude":-80.6481580123579},
'SUN':{"latitude":43.50392605, "longitude":-114.297154199525},
'PSC':{"latitude":46.2635986, "longitude":-119.118901511519},
'IWD':{"latitude":46.5274978, "longitude":-90.1340891492056},
'MCE':{"latitude":37.2849956999999, "longitude":-120.514219717027},
'OGS':{"latitude":44.681759, "longitude":-75.4768},
'ILM':{"latitude":34.27353635, "longitude":-77.9095350717388},
'FIS':{"latitude":52.4636853, "longitude":9.70315499999999},
'HPN':{"latitude":41.0680174, "longitude":-73.7083333382348},
'LCK':{"latitude":39.8132172, "longitude":-82.934199729754},
'TXB':{"latitude":15.246905, "longitude":104.870735},
'IWA':{"latitude":56.941138, "longitude":40.9389727081019},
'BMI':{"latitude":40.4733557, "longitude":-88.9148026461392},
    }

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params_template = {
    "start_date": "2023-07-01",
    "end_date": "2023-12-31",
    "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_100m"]
}

# Initialize a list to store data for all airport locations
all_data = []

# Process weather data for each airport location
for airport_code, location in airport_locations.items():
    # Update latitude and longitude in the request parameters
    params = {**params_template, "latitude": location["latitude"], "longitude": location["longitude"]}
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(3).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "airport_code": [airport_code] * len(hourly_temperature_2m),  # Repeat airport code for each row
        "temperature_2m": hourly_temperature_2m,
        "relative_humidity_2m": hourly_relative_humidity_2m,
        "precipitation": hourly_precipitation,
        "wind_speed_100m": hourly_wind_speed_100m
    }

    all_data.append(hourly_data)

# Combine data for all airport locations into a single DataFrame
all_data_df = pd.concat([pd.DataFrame(data) for data in all_data])

# Split the 'date' column into 'date' and 'hour' columns
all_data_df['date'] = pd.to_datetime(all_data_df['date'])  # Convert 'date' column to datetime
all_data_df['Time'] = all_data_df['date'].dt.hour  # Extract hour from 'date' column
all_data_df['Date'] = all_data_df['date'].dt.date  # Extract date from 'date' column

# Reorder the columns for better readability
all_data_df = all_data_df[['Date', 'Time', 'airport_code', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'wind_speed_100m']]

# Print the combined DataFrame with separate 'date' and 'hour' columns
print(all_data_df)


            Date  Time airport_code  temperature_2m  relative_humidity_2m  \
0     2023-07-01     0          FLA       22.674002             95.248306   
1     2023-07-01     1          FLA       22.524000             95.534431   
2     2023-07-01     2          FLA       22.174002             99.695900   
3     2023-07-01     3          FLA       22.074001            100.000000   
4     2023-07-01     4          FLA       21.574001            100.000000   
...          ...   ...          ...             ...                   ...   
4411  2023-12-31    19          BMI        0.078500             64.028908   
4412  2023-12-31    20          BMI        0.178500             65.281990   
4413  2023-12-31    21          BMI        0.428500             65.587921   
4414  2023-12-31    22          BMI        0.278500             70.697693   
4415  2023-12-31    23          BMI        0.078500             78.470383   

      precipitation  wind_speed_100m  
0               0.0        11.525623

In [3]:
import os
# Specify the directory where you want to save the CSV file
output_directory = '/Users/gms/Desktop'

# Create the full file path for the CSV file
csv_file_path = os.path.join(output_directory, 'weather_data.csv')

# Save the DataFrame to the CSV file
all_data_df.to_csv(csv_file_path, index=False)

print(f"CSV file saved to: {csv_file_path}")

CSV file saved to: /Users/gms/Desktop/weather_data.csv


In [4]:
print(all_data_df)

            Date  Time airport_code  temperature_2m  relative_humidity_2m  \
0     2023-07-01     0          FLA       22.674002             95.248306   
1     2023-07-01     1          FLA       22.524000             95.534431   
2     2023-07-01     2          FLA       22.174002             99.695900   
3     2023-07-01     3          FLA       22.074001            100.000000   
4     2023-07-01     4          FLA       21.574001            100.000000   
...          ...   ...          ...             ...                   ...   
4411  2023-12-31    19          BMI        0.078500             64.028908   
4412  2023-12-31    20          BMI        0.178500             65.281990   
4413  2023-12-31    21          BMI        0.428500             65.587921   
4414  2023-12-31    22          BMI        0.278500             70.697693   
4415  2023-12-31    23          BMI        0.078500             78.470383   

      precipitation  wind_speed_100m  
0               0.0        11.525623

In [5]:
# check null data
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965120 entries, 0 to 4415
Data columns (total 7 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Date                  object 
 1   Time                  int64  
 2   airport_code          object 
 3   temperature_2m        float32
 4   relative_humidity_2m  float32
 5   precipitation         float32
 6   wind_speed_100m       float32
dtypes: float32(4), int64(1), object(2)
memory usage: 90.0+ MB


In [9]:
#put into cloud storatge (create and upload)
CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=teamprojectcis9440;AccountKey=gD9xnf//7J7h70d5tgNEEZDOQ+XUHxM1Se78wNXVzKYrfueXt0qZ8dYgqSLjGlKBBriI6Jwv+614+ASt60ATjQ==;EndpointSuffix=core.windows.net"
CONTAINER_AZURE = 'tsaweather'
blob_name = "weatherdata Jul1-Dec31.csv"
# Convert DataFrame to CSV
output = StringIO()
all_data_df.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded weatherdata Jul1-Dec31.csv to Azure Blob Storage in container tsaweather.
